In [ ]:
"""
Cliente Agente para Recomendação de Filmes
Conversa com LLama 3 (via LM Studio) usando MCP
"""

import asyncio
import json
from typing import List, Dict, Any
from openai import OpenAI
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# ============================================
# CONFIGURAÇÕES
# ============================================

# LM Studio API (compatível com OpenAI)
LM_STUDIO_URL = "http://localhost:1234/v1"
MODELO = "local-model"  # LM Studio usa esse nome genérico

# System prompt para o agente
SYSTEM_PROMPT = """Você é um assistente especializado em recomendar filmes.

Você tem acesso a um banco de dados de filmes através das seguintes ferramentas:
- buscar_filmes: Busca filmes por descrição, gênero ou tema
- listar_todos_filmes: Lista todos os filmes disponíveis
- estatisticas_banco: Mostra estatísticas do banco

IMPORTANTE:
- Sempre use as ferramentas para buscar informações antes de responder
- Seja amigável e conversacional
- Faça perguntas se não entender o que o usuário quer
- Explique POR QUE você está recomendando cada filme
- Use as sinopses dos filmes para contextualizar suas recomendações

Exemplos de uso:
- Usuário: "Quero um filme de ação"
  → Use buscar_filmes(query="filme de ação")

- Usuário: "Algo engraçado"
  → Use buscar_filmes(query="comédia engraçada")

- Usuário: "Quais filmes você tem?"
  → Use listar_todos_filmes()"""

# ============================================
# CLIENTE AGENTE
# ============================================

class AgenteFilmes:
    """Cliente que conecta LLama 3 com o servidor MCP"""

    def __init__(self):
        self.client_openai = OpenAI(
            base_url=LM_STUDIO_URL,
            api_key="lm-studio"  # LM Studio não valida a key
        )
        self.session = None
        self.historico = [
            {"role": "system", "content": SYSTEM_PROMPT}
        ]

    async def inicializar_mcp(self):
        """Conecta ao servidor MCP"""
        print("🔌 Conectando ao servidor MCP...")

        server_params = StdioServerParameters(
            command="python",
            args=["mcp_server.py"],
            env=None
        )

        stdio_transport = await stdio_client(server_params)
        self.stdio, self.write = stdio_transport

        self.session = ClientSession(self.stdio, self.write)
        await self.session.initialize()

        # Lista ferramentas disponíveis
        tools_response = await self.session.list_tools()
        print(f"✓ {len(tools_response.tools)} ferramentas disponíveis:")
        for tool in tools_response.tools:
            print(f"  • {tool.name}")

        return tools_response.tools

    def _converter_tools_para_openai(self, tools) -> List[Dict]:
        """Converte tools MCP para formato OpenAI"""
        tools_openai = []

        for tool in tools:
            tools_openai.append({
                "type": "function",
                "function": {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": tool.inputSchema
                }
            })

        return tools_openai

    async def _executar_tool(self, tool_name: str, arguments: dict) -> str:
        """Executa uma tool via MCP"""
        print(f"\n🔧 Executando: {tool_name}")
        print(f"   Argumentos: {arguments}")

        result = await self.session.call_tool(tool_name, arguments)

        # Extrai texto do resultado
        if result.content and len(result.content) > 0:
            return result.content[0].text

        return "Nenhum resultado retornado"

    async def chat(self, mensagem_usuario: str, tools: List[Dict]) -> str:
        """
        Processa uma mensagem do usuário

        Args:
            mensagem_usuario: Mensagem do usuário
            tools: Lista de ferramentas disponíveis

        Returns:
            Resposta do agente
        """
        # Adiciona mensagem do usuário ao histórico
        self.historico.append({
            "role": "user",
            "content": mensagem_usuario
        })

        max_iteracoes = 5
        iteracao = 0

        while iteracao < max_iteracoes:
            iteracao += 1

            # Chama LLama 3
            print(f"\n💭 Pensando... (iteração {iteracao})")

            response = self.client_openai.chat.completions.create(
                model=MODELO,
                messages=self.historico,
                tools=tools,
                tool_choice="auto",
                temperature=0.7,
                max_tokens=1000
            )

            message = response.choices[0].message

            # Se não tem tool calls, retorna a resposta
            if not message.tool_calls:
                resposta_final = message.content
                self.historico.append({
                    "role": "assistant",
                    "content": resposta_final
                })
                return resposta_final

            # Processa tool calls
            self.historico.append({
                "role": "assistant",
                "content": message.content,
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": "function",
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments
                        }
                    }
                    for tc in message.tool_calls
                ]
            })

            # Executa cada tool call
            for tool_call in message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                # Executa via MCP
                resultado = await self._executar_tool(function_name, function_args)

                print(f"✓ Resultado recebido ({len(resultado)} caracteres)")

                # Adiciona resultado ao histórico
                self.historico.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": resultado
                })

        return "⚠ Número máximo de iterações atingido"

    def limpar_historico(self):
        """Limpa o histórico de conversa"""
        self.historico = [
            {"role": "system", "content": SYSTEM_PROMPT}
        ]
        print("🗑️ Histórico limpo")


# ============================================
# INTERFACE CLI
# ============================================

async def main():
    """Interface de linha de comando"""
    print("="*60)
    print("🎬 AGENTE DE RECOMENDAÇÃO DE FILMES")
    print("="*60)
    print("\nInicializando...")

    # Cria agente
    agente = AgenteFilmes()

    # Conecta ao MCP
    try:
        tools = await agente.inicializar_mcp()
        tools_openai = agente._converter_tools_para_openai(tools)
    except Exception as e:
        print(f"\n❌ Erro ao conectar ao servidor MCP: {e}")
        print("\nVerifique:")
        print("  1. O servidor está rodando? (python mcp_server.py)")
        print("  2. O Docker PostgreSQL está ativo?")
        print("  3. O modelo de embeddings foi baixado?")
        return

    print("\n" + "="*60)
    print("✓ Agente pronto!")
    print("="*60)
    print("\n💡 Dicas:")
    print("  • Digite 'sair' para encerrar")
    print("  • Digite 'limpar' para limpar histórico")
    print("  • Digite 'stats' para ver estatísticas")
    print("\n" + "="*60)

    # Loop de conversa
    while True:
        try:
            # Input do usuário
            print("\n👤 Você:", end=" ")
            mensagem = input().strip()

            if not mensagem:
                continue

            # Comandos especiais
            if mensagem.lower() in ['sair', 'exit', 'quit']:
                print("\n👋 Até logo!")
                break

            if mensagem.lower() == 'limpar':
                agente.limpar_historico()
                continue

            if mensagem.lower() == 'stats':
                resultado = await agente._executar_tool("estatisticas_banco", {})
                print(f"\n📊 {resultado}")
                continue

            # Processa mensagem
            resposta = await agente.chat(mensagem, tools_openai)

            # Exibe resposta
            print(f"\n🤖 Agente: {resposta}")

        except KeyboardInterrupt:
            print("\n\n👋 Até logo!")
            break
        except Exception as e:
            print(f"\n❌ Erro: {e}")
            print("Continuando...")


# ============================================
# TESTE SIMPLES
# ============================================

async def teste_simples():
    """Teste rápido do agente"""
    print("🧪 Teste do agente\n")

    agente = AgenteFilmes()
    tools = await agente.inicializar_mcp()
    tools_openai = agente._converter_tools_para_openai(tools)

    # Testa algumas queries
    queries = [
        "Quero um filme de ação emocionante",
        "Algo mais tranquilo, uma comédia",
        "Quais filmes você tem sobre espionagem?"
    ]

    for query in queries:
        print(f"\n{'='*60}")
        print(f"👤 {query}")
        print(f"{'='*60}")

        resposta = await agente.chat(query, tools_openai)
        print(f"\n🤖 {resposta}\n")

        await asyncio.sleep(1)


if __name__ == "__main__":
    import sys

    if len(sys.argv) > 1 and sys.argv[1] == "teste":
        asyncio.run(teste_simples())
    else:
        asyncio.run(main())